In [1]:
import logging
import os
import pickle
from pathlib import Path
from citylearn.citylearn import CityLearnEnv
from citylearn.utilities import read_json

In [2]:
LOGGER = logging.getLogger(__name__)
log_filepath = Path(f'/Users/kingsleyenweye/Desktop/INTELLIGENT_ENVIRONMENT_LAB/citylearn/buildsys_2022/simulation.log')
LOGGER.setLevel(logging.DEBUG)
handler = logging.FileHandler(log_filepath,mode='w')
formatter = logging.Formatter('%(asctime)s: %(message)s')
handler.setFormatter(formatter)
LOGGER.addHandler(handler)
data_directory = Path('/Users/kingsleyenweye/Desktop/INTELLIGENT_ENVIRONMENT_LAB/citylearn/buildsys_2022/data')
root_directory = Path('/Users/kingsleyenweye/Desktop/INTELLIGENT_ENVIRONMENT_LAB/epri_data/data/citylearn_challenge_2022_phase_3/')
schema_filepath = os.path.join(root_directory,'schema.json')
schema = read_json(schema_filepath)
schema['root_directory'] = root_directory.as_posix()
schema['episodes'] = 20
schema['agent'] = {
    'type':'citylearn.agents.sac.SAC',
    'attributes':{
        'hidden_dimension':[256,256],
        'discount':0.99,
        'tau':0.005,
        'lr':0.003,
        'batch_size':256,
        'replay_buffer_capacity':100_000,
        'start_training_time_step':6_000,
        'end_exploration_time_step':7_000,
        'deterministic_start_time_step':157_680,
        'action_scaling_coef':0.5,
        'reward_scaling':5.0,
        'update_per_time_step':2,
    }
}
schema['reward_function'] = 'reward.MultiObjectiveReward'
env = CityLearnEnv(schema)
agents = env.load_agents()

In [3]:
for episode in range(schema['episodes']):
    observations_list = env.reset()

    while not env.done:
        actions_list = []

        # select actions
        for agent, observations in zip(agents, observations_list):
            if agent.action_dimension > 0:
                actions = agent.select_actions(observations)
                
            else:
                actions = []
            
            actions_list.append(actions)

        # apply actions to citylearn_env
        next_observations_list, reward_list, _, _ = env.step(actions_list)

        # update
        for agent, observations, actions, reward, next_observations in zip(agents, observations_list, actions_list, reward_list, next_observations_list):
            if agent.action_dimension > 0:
                agent.add_to_buffer(observations, actions, reward, next_observations, done=env.done)
            else:
                continue

        observations_list = [o for o in next_observations_list]

        LOGGER.debug(
            f'Time step: {env.time_step}/{env.time_steps - 1},'\
                f' Episode: {episode}/{schema["episodes"] - 1},'\
                    f' Actions: {actions_list},'\
                        f' Rewards: {reward_list}'
        )

    with open(os.path.join(data_directory,f'simulation_1_episode_{int(episode)}.pkl'),'wb') as f:
        pickle.dump({'env':env,'agents':agents},f)

OSError: [Errno 28] No space left on device